Data sources:
- [Vehicles from Traficom](https://tieto.traficom.fi/en/datatraficom/open-data?toggle=Open%20data%20for%20vehicles)
- [Municipalities from Statistics Finland](https://stat.fi/en/luokitukset/kunta/)
- [Geographic from MAPOG](https://gisdata.mapog.com/finland/administrative_boundaries_level8_polygon)

In [1]:
# Municipalities data
from src.data_import import get_municipalities
municipalities = get_municipalities()

display(municipalities)

{'020': 'Akaa',
 '005': 'Alajärvi',
 '009': 'Alavieska',
 '010': 'Alavus',
 '016': 'Asikkala',
 '018': 'Askola',
 '019': 'Aura',
 '035': 'Brändö',
 '043': 'Eckerö',
 '046': 'Enonkoski',
 '047': 'Enontekiö',
 '049': 'Espoo',
 '050': 'Eura',
 '051': 'Eurajoki',
 '052': 'Evijärvi',
 '060': 'Finström',
 '061': 'Forssa',
 '062': 'Föglö',
 '065': 'Geta',
 '069': 'Haapajärvi',
 '071': 'Haapavesi',
 '072': 'Hailuoto',
 '074': 'Halsua',
 '075': 'Hamina',
 '076': 'Hammarland',
 '077': 'Hankasalmi',
 '078': 'Hanko',
 '079': 'Harjavalta',
 '081': 'Hartola',
 '082': 'Hattula',
 '086': 'Hausjärvi',
 '111': 'Heinola',
 '090': 'Heinävesi',
 '091': 'Helsinki',
 '097': 'Hirvensalmi',
 '098': 'Hollola',
 '102': 'Huittinen',
 '103': 'Humppila',
 '105': 'Hyrynsalmi',
 '106': 'Hyvinkää',
 '108': 'Hämeenkyrö',
 '109': 'Hämeenlinna',
 '139': 'Ii',
 '140': 'Iisalmi',
 '142': 'Iitti',
 '143': 'Ikaalinen',
 '145': 'Ilmajoki',
 '146': 'Ilomantsi',
 '153': 'Imatra',
 '148': 'Inari',
 '149': 'Ingå',
 '151': 'Isojok

In [2]:
# Vehicles data
from src.data_import import get_vehicles
vehicles = get_vehicles()

display(vehicles.dtypes)
display(vehicles.shape)
display(vehicles.head(10))

registration_date    object
intro_date           object
color                object
driving_force        object
is_hybrid            object
maker_text           object
municipality         object
odometer             object
dtype: object

(2836294, 8)

,registration_date,intro_date,color,driving_force,is_hybrid,maker_text,municipality,odometer
0,1984-07-09,19840000,1,01,NaN,Ford,740,NaN
1,1990-05-08,19900508,9,01,NaN,Citroen,091,NaN
2,2003-10-02,20031002,6,01,NaN,Honda,837,284104
3,2006-03-17,20060317,Y,01,NaN,Toyota,989,155944
4,2007-01-05,20070105,2,02,NaN,Toyota,694,2692651
5,1996-03-14,19960314,5,01,NaN,Nissan,777,262907
6,2003-07-01,20030701,8,01,NaN,Honda,851,262915
7,2000-03-24,20000324,6,01,NaN,Nissan,250,186269
8,2005-10-25,20051025,8,01,NaN,BMW,755,205732
9,2001-06-06,20010606,Y,01,NaN,Hyundai,407,389645


In [3]:
import copy
import importlib
import src.data_cleaning

importlib.reload(src.data_cleaning)

df = vehicles.copy(deep=True)
mun = copy.deepcopy(municipalities)

(df, mun) = src.data_cleaning.clean(df, mun)

# Check which high count Other labeled makers are missing from mapping
other_makers = df[df["maker"] == "Other"]
maker_counts = other_makers["maker_text"].value_counts()
top_maker_rows = other_makers[other_makers["maker_text"].isin(maker_counts.index)]
top_maker_counts_in_top_rows = top_maker_rows["maker_text"].value_counts().nlargest(10)
display(top_maker_counts_in_top_rows)

df.reset_index(inplace=True, drop=True)

# NA check
na_rows = df[df[df.columns].isna().any(axis=1)]

# Sanity checks
display(df.dtypes)
display(df.shape) # Same size as before
display(f"Min year: {df["registration_year"].min()}", f"Max year: {df["registration_year"].max()}")
display(na_rows) # Should have none

maker_text
capron       2712
adria        2626
mg           1667
pontiac      1388
dethleffs    1327
hymer        1041
buick         920
weinsberg     913
plymouth      880
hobby         760
Name: count, dtype: int64

color                object
driving_force        object
maker_text           object
municipality         object
odometer              Int32
registration_year     Int16
maker                object
dtype: object

(2836294, 7)

'Min year: 1979'

'Max year: 2024'

,color,driving_force,maker_text,municipality,odometer,registration_year,maker


In [4]:
# Final data
import json
import importlib
import os
import src.data_cleaning
import src.data_import
import src.data_validation


importlib.reload(src.data_cleaning)
importlib.reload(src.data_validation)

date = src.data_import.get_date()
final = src.data_cleaning.generate(df, mun, date)
valid = src.data_validation.validate(final, mun)

if valid:
    path = os.path.join(os.path.join(os.getcwd(), "data.json"))
    file = json.dumps(final, indent=2, ensure_ascii=False)
    if os.path.isfile(path):
        os.remove(path)
    with open(path, "w", encoding="utf-8") as fh:
        fh.write(file)

display(final.keys())
display(final["municipalities"][0])

dict_keys(['date', 'municipalities'])

{'code': '020',
 'name': 'Akaa',
 'countByDrivingForce': {'diesel': 2781,
  'electricity': 311,
  'hybrid': 769,
  'other': 74,
  'petrol': 6282},
 'countByColor': {'black': 1428,
  'blue': 1444,
  'brown': 738,
  'green': 430,
  'grey': 2186,
  'other': 282,
  'red': 1480,
  'silver': 676,
  'white': 1553},
 'countByRegistrationYear': {'1979': 115,
  '1980': 10,
  '1981': 10,
  '1982': 15,
  '1983': 25,
  '1984': 33,
  '1985': 46,
  '1986': 56,
  '1987': 76,
  '1988': 86,
  '1989': 93,
  '1990': 104,
  '1991': 40,
  '1992': 43,
  '1993': 42,
  '1994': 41,
  '1995': 51,
  '1996': 74,
  '1997': 76,
  '1998': 107,
  '1999': 146,
  '2000': 179,
  '2001': 166,
  '2002': 248,
  '2003': 337,
  '2004': 329,
  '2005': 410,
  '2006': 403,
  '2007': 371,
  '2008': 456,
  '2009': 304,
  '2010': 417,
  '2011': 459,
  '2012': 390,
  '2013': 350,
  '2014': 402,
  '2015': 432,
  '2016': 449,
  '2017': 432,
  '2018': 460,
  '2019': 441,
  '2020': 374,
  '2021': 362,
  '2022': 329,
  '2023': 309,
  '20